### Should this Loan Be Approved Dataset
- Should this Loan Be Approved Dataset: https://www.tandfonline.com/doi/epdf/10.1080/10691898.2018.1434342?needAccess=true&role=button
- 25 features, output label ~ MIS_Status (loan at a high risk of default, or high risk of being paid off)

### Import the libraries we'll need

In [1]:
# Import the libraries we'll use below.
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style

import tensorflow as tf
from tensorflow import keras
from keras import metrics

%matplotlib inline

### Import and Visualize SBA Loan Data

In [3]:
sba = pd.read_csv('../datasets/SBAnational.csv', delimiter=',')

C:\Users\andre\anaconda3\envs\w207\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
print(sba['MIS_Status'].value_counts(normalize=True))
print('\n')
sba.head()

P I F     0.824383
CHGOFF    0.175617
Name: MIS_Status, dtype: float64




,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [7]:
sba = sba.reindex(np.random.permutation(sba.shape[0]))
display(sba)

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
397545,3832235008,930 ENTERPRISES LIMITED INC,PHOENIX,AZ,85040,"SUPERIOR FINANCIAL GROUP, LLC",CA,321920,28-Dec-09,2010,...,N,NaN,8-Aug-13,28-Dec-09,"$7,500.00",$0.00,CHGOFF,"$5,882.00","$7,500.00","$6,750.00"
373015,3613164004,STEPHEN K WILSON AGENCY LLC,SPOKANE,WA,99205,WELLS FARGO BANK NATL ASSOC,SD,524210,2-Mar-00,2000,...,0,N,NaN,30-Apr-00,"$132,000.00",$0.00,P I F,$0.00,"$132,000.00","$99,000.00"
416758,4027284007,"MARINO PLUMBING & HEATING, INC",BALTIMORE,MD,21228,SUSQUEHANNA BANK,MD,235110,26-Oct-00,2001,...,N,N,NaN,30-Nov-00,"$200,000.00",$0.00,P I F,$0.00,"$200,000.00","$150,000.00"
263592,2844296003,RICK'S AUTO BODY,LOS ANGELES,CA,90027,BANK OF MANHATTAN NATL ASSOC,CA,811121,12-Jun-07,2007,...,Y,N,NaN,31-Jul-07,"$25,000.00",$0.00,P I F,$0.00,"$25,000.00","$12,500.00"
253546,2781644000,MIKE SHEPHERD CHRYSLER,CAMDENTON,MO,65020,BANK OF SULLIVAN,MO,441110,12-Feb-99,1999,...,N,N,NaN,31-Mar-99,"$450,000.00",$0.00,P I F,$0.00,"$475,000.00","$356,250.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692030,7173844000,STREAMLINE MARKETING LLC,MOUNT PLEASANT,SC,29464,BANK OF AMERICA NATL ASSOC,NC,541613,10-Feb-04,2004,...,Y,N,NaN,29-Feb-04,"$50,000.00",$0.00,P I F,$0.00,"$50,000.00","$25,000.00"
176495,2259934001,JEFFREY D PHELPS DMD,ALBANY,OR,97321,WELLS FARGO BANK NATL ASSOC,SD,621210,14-Aug-98,1998,...,0,N,NaN,30-Sep-98,"$221,000.00",$0.00,P I F,$0.00,"$221,000.00","$165,750.00"
537525,5258673001,MFG OF GREENVILLE,GREENVILLE,SC,29607,TRANSAMERICA SMALL BUS. CAPITA,SC,713110,31-Oct-92,1993,...,N,N,NaN,31-Jan-93,"$470,000.00",$0.00,P I F,$0.00,"$470,000.00","$399,500.00"
674611,6909754007,EMBARC.BIZ LLC,HAUPPAUGE,NY,11788,BBCN BANK,CA,514199,12-Nov-03,2004,...,0,N,16-Jul-08,30-Nov-03,"$10,000.00",$0.00,P I F,"$5,500.00","$10,000.00","$8,500.00"


In [11]:
print(sba.columns)

print(len(sba.columns))

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv'],
      dtype='object')
27


In [18]:
print(sba.dtypes)

LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
ApprovalDate          object
ApprovalFY            object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
MIS_Status            object
ChgOffPrinGr          object
GrAppv                object
SBA_Appv              object
dtype: object


In [19]:
print(sba.isnull().sum())

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64
